In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [ ]:
import os
import pandas as pd

def buscar_e_salvar_dados(nome_tabela, mes, ano, colunas='*', limite=10000, caminho_csv=None):
    # Converte colunas para string para a consulta
    if isinstance(colunas, list):
        colunas_str = ", ".join(colunas)
    else:
        colunas_str = colunas  # Supondo que colunas podem ser '*' para todos

    # Prepara strings de data para nomear arquivos
    mes_str = f"{mes:02d}"

    etapa = 1
    offset = 0
    alcançado = False
    while not alcançado:
        print(f'Etapa {etapa} - Buscando dados para {mes}/{ano}')

        consulta = f"""
            SELECT {colunas_str} FROM {nome_tabela}
            WHERE MONTH(DATETIME_) = {mes} AND YEAR(DATETIME_)
            = {ano} ORDER BY `TIMESTAMP_USEC`
            LIMIT {limite} OFFSET {offset};
        """
        
        df = pd.read_sql(consulta, engine)
        print('Dados consultados.')

        # Determina o caminho para salvar
        etapa_str = f"{etapa:02d}"
        nome = f"{ano}{mes_str}_{etapa_str}_machinedata_filtered.csv"
        if caminho_csv is None:
            caminho_completo = os.path.join(os.getcwd(), nome)  # Salva no diretório de trabalho atual
        else:
            caminho_completo = os.path.join(caminho_csv, nome)

        # Salva para CSV
        df.to_csv(caminho_completo, index=False)
        print(f"Arquivo criado em {caminho_completo}. Recuperadas {len(df)} linhas.")
        
        if len(df) < limite:
            alcançado = True
        offset += limite
        etapa += 1



In [ ]:
USUARIO = 'meu_usuario'
SENHA = 'minha_senha'
IP_REMOTO = '100.000.000.001'
DATABASE = 'meu_database'

engine_url = f'mysql+pymysql://{USUARIO}:{SENHA}@{IP_REMOTO}:3306/{DATABASE}'
path = r'C:\Users\...\GitHub\MySQL_Tool\salvar'
engine = create_engine(engine_url)

for mes in ([6,7,8,9,10]):
    colunas = [
    'Column_A',
    'Column_B',
    'Column_C',
    'Timestamp'
    ]
    buscar_e_salvar_dados(nome_tabela='COLHEDORA_DATA', mes=mes, ano=2024, caminho_csv=path, colunas=colunas)
    # buscar_e_salvar_dados(nome_tabela='COLHEDORA_DATA_FILTRADO', mes=mes, ano=2024, caminho_csv=path, colunas=colunas, limite = 100000)
engine.dispose()